In [38]:
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
import warnings
warnings.filterwarnings("ignore")

In [32]:
def GBM_characteristic_function(u, sigma, SIGMA, T, r):
  """Calculate the extended characteristic function of Multivariate GBM.
  Args:
  - u (array): Array of Fourier frequencies.
  - sigma (array): Array of volatilities of each stock.
  - SIGMA (array): Covariance matrix.
  - T (float): Time to maturity.
  - r (float): Risk-free interest rate.
  Returns:
  - phi (complex): Extended characteristic function value.
  """
  d = len(sigma)  # Number of stocks
  # Compute the characteristic function
  phi = np.exp(np.dot(np.multiply(1j * T, u), r * np.ones(d) - 0.5 * np.diag(SIGMA)) - 0.5 * T * np.dot(u, np.dot(SIGMA, u)))
  return phi

def covariance_matrix(sigma, rho):
  """Compute the covariance matrix.
  Args:
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  Returns:
  - SIGMA (array): Covariance matrix.
  """
  SIGMA = np.dot(np.diag(sigma), np.dot(rho, np.diag(sigma)))
  return SIGMA

def fourier_payoff_call_on_min(u):
  """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transform value.
  """
  # Compute the Fourier transform of the payoff function
  denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
  return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Basket put option payoff Fourier transform value.
  """
  # Compute the Fourier transform of the basket put option payoff function
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator / denominator)

def integrand_to_optimize_GBM_call_on_min(R):
  """Calculate the integrand of the GBM to optimize.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value at the origin (u = 0).
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, K))
  y = np.multiply(1j, R)
  phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
  p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
  discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  integrand = discount * phi * p
  return np.real(integrand)  # Real part of the integrand

def integrand_to_optimize_GBM_basket_put(R):
  """Calculate the integrand of the GBM to optimize.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value at the origin (u = 0).
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, d * K))
  y = np.multiply(1j, R)
  phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
  p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
  discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  integrand = discount * phi * p
  return np.real(integrand)  # Real part of the integrand


def fourier_MC_call_on_min_GBM(S0, K, r, T, sigma, SIGMA, N, R, SIGMA_IS, alpha_conf, seed):
  """
  Estimate the price of call on min options using Monte Carlo simulation in Fourier space with importance sampling using Gaussian distribution
  Args:
  - S0 (array): vector of initial stock prices.
  - K (float): strike price.
  - r (float): risk-free interest rate.
  - T (float): time to maturity.
  - sigma (array): volatilities of each stock.
  - SIGMA (array): covariance matrix of Brownian motions.
  - N (int): number of Monte Carlo samples.
  - R (array): vector of damping parameters
  - sigma_IS (float): importance sampling parameters for MC in Fourier space.
  - seed (int): seed for the random generator
  - alpha_conf: confidence level
  Returns:
  - MC_estimate (float): Estimated price of the call on min option.
  - MC_stat_error (float): Statistical error of the Monte Carlo estimation.
  """
  np.random.seed(seed)
  # Number of stocks
  dimension = len(sigma)
  # Logarithm of the element-wise division
  X0 = np.log(np.divide(S0,K))
  # Modified discount factor
  discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
  # Contains Monte Carlo price estimates
  V_list = np.zeros(N)
  # Generate correlated samples
  samples = np.random.multivariate_normal(mean = np.zeros(dimension), cov = SIGMA_IS, size=N)
  multivar_normal = multivariate_normal(mean = np.zeros(dimension), cov = SIGMA_IS)
  # For each sample
  for n in range(N):
    u = samples[n]  # Sample from the standard normal distribution
    y = u + np.multiply(1j, R) # shifting contour of integration by the damping parameters
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Evaluate characteristic function
    # Evaluate Fourier Transformed Payoff function
    p = fourier_payoff_call_on_min(y)
    # Product of Gaussian densities
    gaussian_pdf_eval = multivar_normal.pdf(u)
    # Compute Monte Carlo estimators
    V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / gaussian_pdf_eval)
  # Compute the Monte Carlo estimate
  MC_estimate = np.mean(V_list)
  # Compute the statistical error
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = 1.96 * np.std(V_list) / np.sqrt(N)
  return MC_estimate, MC_stat_error


def fourier_MC_basket_put_GBM(S0, K, r, T, sigma, SIGMA, N, R, SIGMA_IS, alpha_conf, seed):
  """
  Estimate the price of basket put options using Monte Carlo simulation in Fourier space with importance sampling using Gaussian distribution
  Args:
  - S0 (array): vector of initial stock prices.
  - K (float): strike price.
  - r (float): risk-free interest rate.
  - T (float): time to maturity.
  - sigma (array): volatilities of each stock.
  - SIGMA (array): covariance matrix of Brownian motions.
  - N (int): number of Monte Carlo samples.
  - R (array): vector of damping parameters
  - sigma_IS (float): importance sampling parameters for MC in Fourier space.
  - seed (int): seed for the random generator
  - alpha_conf: confidence level
  Returns:
  - MC_estimate (float): Estimated price of the rainbow option.
  - MC_stat_error (float): Statistical error of the Monte Carlo estimation.
  """
  np.random.seed(seed)
  # Number of stocks
  dimension = len(sigma)
  # Logarithm of the element-wise division
  X0 = np.log(np.divide(S0,dimension*K))
  # Modified discount factor
  discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
  # Contains Monte Carlo price estimates
  V_list = np.zeros(N)
  # Generate correlated samples
  samples = np.random.multivariate_normal(mean = np.zeros(dimension), cov = SIGMA_IS, size=N)
  multivar_normal = multivariate_normal(mean = np.zeros(dimension), cov = SIGMA_IS)
  # For each sample
  for n in range(N):
    u = samples[n]  # Sample from the standard normal distribution
    y = u + np.multiply(1j, R) # shifting contour of integration by the damping parameters
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Evaluate characteristic function
    # Evaluate Fourier Transformed Payoff function
    p = fourier_payoff_basket_put(y)
    # Product of Gaussian densities
    gaussian_pdf_eval = multivar_normal.pdf(u)
    # Compute Monte Carlo estimators
    V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / gaussian_pdf_eval)
  # Compute the Monte Carlo estimate
  MC_estimate = np.mean(V_list)
  # Compute the statistical error
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = 1.96 * np.std(V_list) / np.sqrt(N)
  return MC_estimate, MC_stat_error

# Pricing basket put options using MC in the Fourier domain

In [36]:
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#GBM Model Parameters
sigma = 0.4 * np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)


############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R
def rainbow_constraint_2(R):
  return -1 - np.sum(R)
cons = ( {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_call_on_min, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-3.08086818 -3.08086818 -3.08086824 -3.0808682 ]


In [37]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#GBM Model Parameters
sigma = 0.4 * np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### MC parameters ###############
B = 1 # number of batches
N_steps = 1 # exact simulation scheme
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
SIGMA_IS = (1 / T) * la.inv(SIGMA) # Proposed QMC domain transformation
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_call_on_min_GBM(S0, K, r, T, sigma, SIGMA, N, R, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 0.31758 
MC relative statistical error =  0.0032


# Pricing basket put options using MC in the Fourier domain

In [34]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#GBM Model Parameters
sigma = 0.4 * np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': basket_put_constraint},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [2.1147309  2.1147309  2.11473093 2.11473092]


In [35]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0= 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#GBM Model Parameters
sigma = 0.4 * np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### MC parameters ###############
B = 1 # number of batches
N_steps = 1 # exact simulation scheme
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
SIGMA_IS = (1 / T) * la.inv(SIGMA) # Proposed QMC domain transformation
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_basket_put_GBM(S0, K, r, T, sigma, SIGMA, N, R, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 8.24488 
MC relative statistical error =  0.0132
